In [0]:
!pip install gensim

In [0]:
!pip install nltk

In [0]:
from nltk.corpus import stopwords
from nltk.stem import *
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
import gensim

In [0]:
book = open("therepublic.txt", "r")
book = book.read()

In [0]:
stemmer = PorterStemmer()
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
# Tokenize and lemmatize

def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text) :
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
            
    return result

In [27]:
'''
Preview a document after preprocessing
'''
print("Original document: ")
words = []
for word in book.split(' '):
    words.append(word)
print(words[:20])
print("\n\nTokenized and lemmatized document: ")
print(preprocess(book)[:20])

Original document: 
['\ufeffThe', 'Project', 'Gutenberg', 'EBook', 'of', 'The', 'Republic,', 'by', 'Plato\n\nThis', 'eBook', 'is', 'for', 'the', 'use', 'of', 'anyone', 'anywhere', 'at', 'no', 'cost']


Tokenized and lemmatized document: 
['project', 'gutenberg', 'ebook', 'republ', 'plato', 'ebook', 'cost', 'restrict', 'whatsoev', 'copi', 'away', 'term', 'project', 'gutenberg', 'licens', 'includ', 'ebook', 'onlin', 'gutenberg', 'titl']


In [0]:
book = lemmatize_stemming(book)
processed_book = preprocess(book)

In [34]:
'''
Preview 'processed_book'
'''
print(processed_book[:20])

['project', 'gutenberg', 'ebook', 'republ', 'plato', 'ebook', 'cost', 'restrict', 'whatsoev', 'copi', 'away', 'term', 'project', 'gutenberg', 'licens', 'includ', 'ebook', 'onlin', 'gutenberg', 'titl']


In [0]:
'''
Create a dictionary from 'processed_docs' containing the number of times a word appears 
in the training set using gensim.corpora.Dictionary and call it 'dictionary'
'''
dictionary = gensim.corpora.Dictionary([word.split() for word in processed_book])

In [36]:
'''
Checking dictionary created
'''
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 project
1 gutenberg
2 ebook
3 republ
4 plato
5 cost
6 restrict
7 whatsoev
8 copi
9 away
10 term


In [0]:
'''
OPTIONAL STEP
Remove very rare and very common words:

- words appearing less than 15 times
- words appearing in more than 10% of all documents
'''
dictionary.filter_extremes(no_below=15, no_above=0.1, keep_n= 100000)

In [0]:

'''
Create the Bag-of-words model for each document i.e for each document we create a dictionary reporting how many
words and how many times those words appear. Save this to 'bow_corpus'
'''
bow_corpus = [dictionary.doc2bow(processed_book)]

In [40]:
'''
Preview BOW for our sample preprocessed document
'''
for i in range(len(bow_corpus)):
    print("Word {} (\"{}\") appears {} time.".format(bow_corpus[i][0], 
                                                     dictionary[bow_corpus[i][0]], 
                                                     bow_corpus[i][1]))

KeyError: ignored

In [0]:
# LDA mono-core -- fallback code in case LdaMulticore throws an error on your machine
# lda_model = gensim.models.LdaModel(bow_corpus, 
#                                    num_topics = 10, 
#                                    id2word = dictionary,                                    
#                                    passes = 50)

# LDA multicore 
'''
Train your lda model using gensim.models.LdaMulticore and save it to 'lda_model'
'''
# TODO
lda_model =  gensim.models.LdaMulticore(bow_corpus, 
                                   num_topics = 10, 
                                   id2word = dictionary,                                    
                                   passes = 10,
                                   workers = 2)

In [46]:
'''
For each topic, we will explore the words occuring in that topic and its relative weight
'''
for idx, topic in lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic ))
    print("\n")

Topic: 0 
Words: 0.014*"say" + 0.009*"state" + 0.006*"like" + 0.005*"good" + 0.005*"natur" + 0.004*"true" + 0.004*"think" + 0.004*"certainli" + 0.004*"time" + 0.004*"thing"


Topic: 1 
Words: 0.027*"say" + 0.015*"good" + 0.012*"true" + 0.012*"state" + 0.010*"like" + 0.008*"know" + 0.007*"thing" + 0.007*"life" + 0.006*"natur" + 0.006*"soul"


Topic: 2 
Words: 0.010*"say" + 0.006*"good" + 0.005*"like" + 0.005*"state" + 0.005*"natur" + 0.005*"true" + 0.004*"life" + 0.004*"repli" + 0.004*"think" + 0.004*"thing"


Topic: 3 
Words: 0.009*"say" + 0.008*"state" + 0.008*"good" + 0.006*"true" + 0.006*"like" + 0.005*"natur" + 0.005*"soul" + 0.005*"life" + 0.004*"plato" + 0.004*"think"


Topic: 4 
Words: 0.022*"say" + 0.014*"state" + 0.012*"good" + 0.010*"like" + 0.010*"true" + 0.009*"natur" + 0.007*"life" + 0.006*"soul" + 0.006*"plato" + 0.006*"thing"


Topic: 5 
Words: 0.010*"say" + 0.007*"state" + 0.006*"like" + 0.006*"good" + 0.005*"true" + 0.004*"think" + 0.004*"natur" + 0.004*"mean" + 0.003*